## Libraries

In [1]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')



This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 200/200 [02:22<00:00,  1.40it/s]
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


## Contact

In [4]:
df_sc.columns.tolist()


['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'fielder_2',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'w

In [29]:
df_sc = df_sc.sort_values(by=['game_pk', 'at_bat_number'])

df_sc['on_1b_lagged'] = df_sc['on_1b'].shift(-1)
df_sc['on_2b_lagged'] = df_sc['on_2b'].shift(-1)
df_sc['on_3b_lagged'] = df_sc['on_3b'].shift(-1)

df_sc['is_on_1b'] = np.where(df_sc['on_1b'].isnull(), 0, 1)
df_sc['is_on_2b'] = np.where(df_sc['on_2b'].isnull(), 0, 1)
df_sc['is_on_3b'] = np.where(df_sc['on_3b'].isnull(), 0, 1)

df_sc['home_run_scored'] = df_sc['post_home_score'] - df_sc['home_score']
df_sc['away_run_scored'] = df_sc['post_away_score'] - df_sc['away_score']
df_sc['total_runs_scored'] = df_sc['home_run_scored'] + df_sc['away_run_scored']

# df_sc_filtered = df_sc[df_sc['type'] == 'X'].sort_values(by=['game_pk', 'at_bat_number'])

# df_sc_filtered[['game_pk', 'at_bat_number', 'inning', 'inning_topbot', 'events', 'on_1b', 'on_2b', 'on_3b', 'on_1b_moved', 'on_2b_moved', 'total_runs_scored']]


In [43]:

df_sc_runner_on_base = df_sc[(df_sc['on_1b'].notnull()) | df_sc['on_2b'].notnull() | df_sc['on_3b'].notnull()].sort_values(by=['game_pk', 'at_bat_number'])
df_sc_runner_on_base[['game_pk', 'at_bat_number', 'inning', 'inning_topbot', 'events', 'batter', 'on_1b_lagged', 'on_1b', 'on_2b_lagged', 'on_2b', 'on_3b_lagged', 'on_3b', 'total_runs_scored', 'launch_speed', 'launch_angle']]


df_sc_runner_on_base['runner_1b_move'] = np.where(
    pd.notna(df_sc_runner_on_base['on_1b']) & pd.notna(df_sc_runner_on_base['on_2b_lagged']) & (df_sc_runner_on_base['on_1b'] == df_sc_runner_on_base['on_2b_lagged']), 1,
    np.where(
        pd.notna(df_sc_runner_on_base['on_1b']) & pd.notna(df_sc_runner_on_base['on_3b_lagged']) & (df_sc_runner_on_base['on_1b'] == df_sc_runner_on_base['on_3b_lagged']), 2, 0
    )
)

df_sc_runner_on_base['runner_2b_move'] = np.where(
    pd.notna(df_sc_runner_on_base['on_2b']) & pd.notna(df_sc_runner_on_base['on_3b_lagged']) & (df_sc_runner_on_base['on_2b'] == df_sc_runner_on_base['on_3b_lagged']), 1, 0
)


# df_sc_runner_on_base[(df_sc_runner_on_base['game_pk'] == 744795) & (df_sc_runner_on_base['inning'] == 3) & (df_sc_runner_on_base['inning_topbot'] == "Top")][['game_pk', 'at_bat_number', 'inning', 'inning_topbot', 'events', 'batter', 'on_1b_lagged', 'on_1b', 'on_2b_lagged', 'runner_1b_move', 'on_2b', 'on_3b_lagged', 'runner_2b_move', 'on_3b', 'total_runs_scored', 'launch_speed', 'launch_angle']]
df_sc_runner_on_base_filtered = df_sc_runner_on_base[df_sc_runner_on_base['type'] == 'X']   
df_sc_runner_on_base_filtered[['game_pk', 'at_bat_number', 'inning', 'inning_topbot', 'launch_speed', 'launch_angle', 'events', 'des', 'batter', 'runner_1b_move', 'runner_2b_move', 'on_3b', 'total_runs_scored']]



,game_pk,at_bat_number,inning,inning_topbot,launch_speed,launch_angle,events,des,batter,runner_1b_move,runner_2b_move,on_3b,total_runs_scored
2717,744795,4,1,Top,85.4,-3,force_out,"Salvador Perez grounds into a force out, short...",521692,0,0,<NA>,0
1707,744795,10,2,Top,82.1,61,field_out,MJ Melendez pops out to shortstop Nasim Nuñez.,669004,0,0,<NA>,0
1511,744795,11,2,Top,95.0,36,field_out,Maikel Garcia flies out to center fielder Jaco...,672580,0,0,<NA>,0
2149,744795,13,2,Bot,98.8,24,field_out,Joey Gallo flies out to right fielder Tommy Pham.,608336,0,0,<NA>,0
1994,744795,14,2,Bot,95.8,-28,force_out,"Ildemaro Vargas grounds into a force out, seco...",545121,0,0,<NA>,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,748868,46,6,Bot,81.7,-14,single,Trent Grisham singles on a ground ball to righ...,663757,0,0,<NA>,0
1407,748868,47,6,Bot,71.3,-27,force_out,"Alex Verdugo grounds into a force out, second ...",657077,0,0,592450,1
1154,748868,48,6,Bot,62.4,4,field_out,"Luis Torrens grounds out, second baseman Wyatt...",620443,0,0,<NA>,0
2065,748868,51,7,Top,100.3,8,field_out,Tyrone Taylor lines out sharply to third basem...,621438,0,0,<NA>,0


In [20]:
import pandas as pd
import numpy as np

# Assuming df_sc is your DataFrame
# Filter the data based on specific launch angle and exit velocity
launch_angle = 30  # example value
exit_velocity = 100  # example value

filtered_df = df_sc[(df_sc['launch_angle'] == launch_angle) & (df_sc['launch_speed'] == exit_velocity)]

# Calculate the number of bases moved by runners on each base
def calculate_bases_moved(row):
    bases_moved = {'on_1b': 0, 'on_2b': 0, 'on_3b': 0}
    if pd.notna(row['on_1b_moved']):
        if row['on_1b_moved'] == 1:
            bases_moved['on_1b'] = 1
        elif row['on_1b_moved'] == 2:
            bases_moved['on_1b'] = 2
    if pd.notna(row['on_2b_moved']):
        if row['on_2b_moved'] == 1:
            bases_moved['on_2b'] = 1
    if pd.notna(row['on_3b_moved']):
        if row['on_3b_moved'] == 1:
            bases_moved['on_3b'] = 1
    return pd.Series(bases_moved)

filtered_df[['on_1b_bases_moved', 'on_2b_bases_moved', 'on_3b_bases_moved']] = filtered_df.apply(calculate_bases_moved, axis=1)

# Calculate the probabilities
probabilities = {
    'on_1b': filtered_df['on_1b_bases_moved'].value_counts(normalize=True).to_dict(),
    'on_2b': filtered_df['on_2b_bases_moved'].value_counts(normalize=True).to_dict(),
    'on_3b': filtered_df['on_3b_bases_moved'].value_counts(normalize=True).to_dict()
}

print(f"Probabilities of advancing bases for launch angle {launch_angle} and exit velocity {exit_velocity}:")
print("Runner on 1B:", probabilities['on_1b'])
print("Runner on 2B:", probabilities['on_2b'])
print("Runner on 3B:", probabilities['on_3b'])

Probabilities of advancing bases for launch angle 30 and exit velocity 100:
Runner on 1B: {0: 1.0}
Runner on 2B: {0: 1.0}
Runner on 3B: {0: 1.0}


/var/folders/fl/cwc6990j59q5wy2jkl371h040000gn/T/ipykernel_83444/1932952588.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['on_1b_bases_moved', 'on_2b_bases_moved', 'on_3b_bases_moved']] = filtered_df.apply(calculate_bases_moved, axis=1)
/var/folders/fl/cwc6990j59q5wy2jkl371h040000gn/T/ipykernel_83444/1932952588.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['on_1b_bases_moved', 'on_2b_bases_moved', 'on_3b_bases_moved']] = filtered_df.apply(calculate_bases_moved, axis=1